# Generating a Ensemble Model with a TF-IDF Feature Set

In [13]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1010)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1010)>


False

### Import Training Data

In [10]:
training_data = "../data/train_long_df.csv"
train_df = pd.read_csv(training_data, dtype={'folder': int, 'text': str, 'is_real': int})
train_df.head()

,folder,text,is_real
0,0,The VIRSA (Visible Infrared Survey Telescope A...,1
1,0,The China relay network has released a signifi...,0
2,1,China\nThe goal of this project involves achie...,0
3,1,The project aims to achieve an accuracy level ...,1
4,2,Scientists can learn about how galaxies form a...,1


### Clean Training Data

In [11]:
clean_df = train_df.copy()
print(f"Number of samples before data cleaning: {len(clean_df)}")

# Removing NA rows altogether 
clean_df.dropna(subset=["text"], inplace=True)
print(f"Number of samples before data cleaning: {len(clean_df)}")

Number of samples before data cleaning: 190
Number of samples before data cleaning: 188


### Feature Engineering

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
punct_table = str.maketrans('', '', string.punctuation)

def preprocess(self, text):
    text = text.lower()
    text = text.translate(self.punct_table)
    tokens = [self.lemmatizer.lemmatize(word)
                for word in text.split()
                if word not in self.stop_words]

    return ' '.join(tokens)

clean_df['clean_text'] = clean_df['text'].apply(preprocess)
clean_df['text_length'] = clean_df['text'].apply(len)
clean_df['word_count'] = clean_df['text'].apply(lambda x: len(x.split()))
clean_df['avg_word_length'] = clean_df['text'].apply(
    lambda x: np.mean([len(w) for w in x.split()]) if len(x.split()) > 0 else 0)
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=10000,
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_tfidf = vectorizer.fit_transform(df['clean_text'])
X_tfidf.head()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/shaemckenna/nltk_data'
    - '/Users/shaemckenna/Programs/real_or_fake/rof/nltk_data'
    - '/Users/shaemckenna/Programs/real_or_fake/rof/share/nltk_data'
    - '/Users/shaemckenna/Programs/real_or_fake/rof/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
